<div style="text-align: center;">
<h1>NFL Elo System</h1>
</div>

## 1. Overview

- Every team starts with a base Elo rating (e.g., 1500).
- After each game, Elo ratings are updated based on:
  - Who won/lost (or tied)
  - The expected outcome (based on pre-game Elo)
  - The margin of victory
  - Home field advantage
- Upsets and blowouts cause bigger Elo changes.
- The bigger the surprise or the margin, the bigger the Elo change.

<br></br>

---

<br></br>

## 2. Step-by-Step

We'll walk through the Elo update for a single game, with formulas and Python code for each step.

### 2.1. Expected Score Calculation

The expected score is the probability each team has to win, based on their Elo ratings and home field advantage.

<br></br>

$$
\text{expected\_home} = \frac{1}{1 + 10^{\frac{(\text{away\_elo} - (\text{home\_elo} + \text{hfa}))}{400}}}
$$

$$
\text{expected\_away} = \frac{1}{1 + 10^{\frac{((\text{home\_elo} + \text{hfa}) - \text{away\_elo})}{400}}}
$$


In [ ]:
# Example Elo ratings and home field advantage
home_elo = 1500
away_elo = 1500
hfa = 2.5  # Home field advantage

expected_home = 1 / (1 + 10 ** ((away_elo - (home_elo + hfa)) / 400))
expected_away = 1 / (1 + 10 ** (((home_elo + hfa) - away_elo) / 400))

print(f"Expected home win probability: {expected_home:.3f}")
print(f"Expected away win probability: {expected_away:.3f}")


### 2.2. Actual Result and Margin

- If home team wins: `actual_home = 1`, `actual_away = 0`
- If away team wins: `actual_home = 0`, `actual_away = 1`
- If tie: `actual_home = 0.5`, `actual_away = 0.5`

The margin of victory is used to scale the Elo change.


In [ ]:
# Example game result
home_score = 24
away_score = 17

if home_score > away_score:
    actual_home, actual_away = 1, 0
    score_diff = home_score - away_score
elif away_score > home_score:
    actual_home, actual_away = 0, 1
    score_diff = away_score - home_score
else:
    actual_home, actual_away = 0.5, 0.5
    score_diff = 0

print(f"Actual home: {actual_home}, Actual away: {actual_away}, Margin: {score_diff}")


### 2.3. K-Factor Adjustment (Margin of Victory)

The K-factor is scaled up for bigger wins:

<br></br>

$$
\text{k\_adj\_factor} = 1 + \frac{\log(|\text{score\_diff}| + 1)}{10}
$$

In [ ]:
import numpy as np
k_adj_factor = 1 + np.log(abs(score_diff) + 1) / 10
print(f"K-factor adjustment: {k_adj_factor:.3f}")

### 2.4. Elo Update Step

The Elo ratings are updated for both teams:

<br></br>

$$
\text{new\_elo} = \text{old\_elo} + K \times \text{k\_adj\_factor} \times (\text{actual} - \text{expected})
$$


In [ ]:
K = 20  # base K-factor
new_home_elo = home_elo + K * k_adj_factor * (actual_home - expected_home)
new_away_elo = away_elo + K * k_adj_factor * (actual_away - expected_away)

print(f"New home Elo: {new_home_elo:.2f}")
print(f"New away Elo: {new_away_elo:.2f}")
print(f"Home Elo change: {new_home_elo - home_elo:+.2f}")
print(f"Away Elo change: {new_away_elo - away_elo:+.2f}")


<br></br>

---

<br></br>

## 3. Changing Parameters

This Elo formula is **very standard** and is the classic approach for rating systems in chess, sports, and more. However, **different models can produce different results** because of several key factors:

#### 1. **Parameter Choices**
- **K-factor:** Some models use a higher or lower K (how fast Elo changes). A higher K makes ratings more volatile.
- **Home Field Advantage (HFA):** The value for HFA can vary (e.g., 2.5, 3.0, or even 0).
- **Margin of Victory Adjustment:** Some models use a different formula or none at all for scaling Elo by the score difference.

#### 2. **Initialization**
- **Starting Elo:** Some models start all teams at 1500, others use historical performance or different baselines.

#### 3. **Game Selection**
- **Which games are included?** Some models use only regular season, others include playoffs, preseason, or even international games.
- **How far back do they go?** Some use only recent years, others use decades of data.

#### 4. **Custom Tweaks**
- Some models add extra adjustments for:
  - **Rest days**
  - **Travel distance**
  - **Injuries**
  - **Weather**
  - **Special weighting for recent games**
- Some use a **different conversion factor** (e.g., 400 in the denominator, or a different base for the exponent).

#### 5. **Post-Processing**
- Some models “regress” ratings toward the mean in the offseason.
- Some blend Elo with other power rankings or stats.

#### 6. **Randomness and Luck**
- Even with the same formula, Elo will diverge over time if you use different data, or if you reset at different points.

#### **Summary Table**

| Factor                        | Example Variation                | Effect on Results         |
|-------------------------------|----------------------------------|--------------------------|
| K-factor                      | 20 vs 30 vs 10                   | More/less rating movement|
| HFA                           | 2.5 vs 3.0 vs 0                  | Home teams more/less favored|
| Margin of Victory             | Log formula vs none vs linear    | Blowouts matter more/less|
| Data selection                | Only regular season vs all games | Different history        |
| Custom tweaks                 | Rest, injuries, etc.             | More/less realism        |
| Initialization                | 1500 vs custom                   | Early season differences |

#### **In summary:**
- The **core Elo formula is standard**, but **small changes in parameters, data, and tweaks** can lead to different results.
- That’s why you’ll see different Elo ratings and predictions from FiveThirtyEight, ESPN, betting sites, and your own model—even if they all use “Elo.”

<br></br>

---

<br></br>

## 4. Power Rankings to Spreads and Moneylines

This section explains how to convert Elo ratings into point spreads and moneyline odds for NFL games, step by step.

#### 1. Elo Power Rankings
- Each team has an Elo rating (higher = stronger).

#### 2. Calculate Elo Difference for a Matchup
- Adjust home team Elo for home field advantage (HFA):

$$
\text{home\_elo\_adj} = \text{home\_elo} + \text{HFA}
$$

- Elo difference:

$$
\text{elo\_diff} = \text{home\_elo\_adj} - \text{away\_elo}
$$

#### 3. Convert Elo Difference to Point Spread
- Use a scaling factor (commonly 25 Elo points = 1 point):

$$
\text{point\_spread} = \frac{\text{elo\_diff}}{25}
$$

#### 4. Convert Point Spread to Win Probability
- Use a normal distribution (NFL historical std ≈ 13.8):

$$
\text{implied\_prob\_home\_cover} = \text{norm.cdf}(0, \text{loc}=-\text{point\_spread}, \text{scale}=13.8)
$$

#### 5. Convert Win Probability to Moneyline Odds
- Adjust for vig (bookmaker margin):

$$
\text{odds\_home} = \text{implied\_prob\_home\_cover} \times \text{vig}
$$
$$
\text{odds\_away} = (1 - \text{implied\_prob\_home\_cover}) \times \text{vig}
$$

- Convert to American odds:

If $p \geq 1$: $-10000$  
If $p > 0.5$: $-100 \times \frac{p}{1-p}$  
Else: $\frac{1-p}{p} \times 100$

#### 6. Output
- For each matchup, you now have:
  - **Point Spread** (e.g., PHI -3.5)
  - **Home Moneyline** (e.g., -165)
  - **Away Moneyline** (e.g., +140)


#### **Summary Table**

| Step                | What it does                                 | Example Output         |
|---------------------|----------------------------------------------|-----------------------|
| Elo Ratings         | Team strength numbers                        | PHI: 1607, DAL: 1523  |
| Elo Difference      | Adjust for home field, get matchup diff      | PHI +2.5 - DAL        |
| Point Spread        | Elo diff / 25                                | PHI -3.5              |
| Win Probability     | Use normal distribution                      | PHI 60%               |
| Moneyline Odds      | Convert probability to American odds         | PHI -165, DAL +140    |

---

**In short:**
- Elo → Elo difference → Point spread → Win probability → Moneyline odds

You can use these formulas and steps to generate betting lines from Elo ratings for any matchup.


In [ ]:
# Example: From Elo to Spread and Moneyline Odds
from scipy.stats import norm

# Example Elo ratings and home field advantage
home_elo = 1607
away_elo = 1523
hfa = 2.5
vig = 1.04

# 1. Adjust for home field
home_elo_adj = home_elo + hfa
elo_diff = home_elo_adj - away_elo

# 2. Convert Elo difference to point spread
point_spread = elo_diff / 25
print(f"Model point spread (home): {point_spread:+.2f}")

# 3. Convert point spread to win probability
implied_prob_home_cover = norm.cdf(0, loc=-point_spread, scale=13.8)

# 4. Adjust for vig
odds_home = implied_prob_home_cover * vig
odds_away = (1 - implied_prob_home_cover) * vig

def to_american_odds(prob):
    if prob >= 1:
        return -10000
    elif prob > 0.5:
        return int(-100 * (prob / (1 - prob)))
    else:
        return int((1 - prob) / prob * 100)

home_moneyline = to_american_odds(odds_home)
away_moneyline = to_american_odds(odds_away)

print(f"Home moneyline: {home_moneyline:+}")
print(f"Away moneyline: {away_moneyline:+}")
